In [ ]:
# draft code used to be in class MyLogger(Logger)'s __init()__ method
# used to set up initial history keys
num_output_layers = 6

self.num_output_layers = num_output_layers
        history_per_output_layer = {
            # per epoch
            'loss':[],
            'acc':[],
            'f1_score':[],
            # these 2 will be replaced with step_y_pred and step_y_true respectively
            'y_pred':[],
            'y_true':[],
        } 

def pre_suf_fix_dict(dict, fix_val='output_layer', prefix=True):
    return {f"{fix_val}_{key}" if prefix else f"{key}_{fix_val}" : value for key, value in dict.items()}

# to create val/test prefixed keys
history_per_output_layer.update(
    **pre_suf_fix_dict(history_per_output_layer, fix_val='val', prefix=True),
    **pre_suf_fix_dict(history_per_output_layer, fix_val='test', prefix=True),
)
# to create step_ prefixed keys
history_per_output_layer.update(
    **pre_suf_fix_dict(history_per_output_layer, fix_val='step', prefix=True),
)
# removing y_pred, y_true and keeping step_y_pred and step_y_true
history_per_output_layer.pop('y_pred')
history_per_output_layer.pop('y_true')

# fix_val == suffix without "_"; to provide flexibility in using fix_val as prefix as well (if we wanted to!)
self.out_layers_fix_val = 'output_layer' 

# pre_suf_fix_dict function is used to create unique key names to merge the `num_output_layers` histories into one history dictionary
history_ol_lists = [pre_suf_fix_dict(history_per_output_layer, 
                                    fix_val=f'{self.out_layers_fix_val}_{i}', 
                                    prefix=False) 
                    for i in range(self.num_output_layers)] # ol == output layers

# 'loss' here is for the final loss calculated by the loss aggregator function which uses all `num_output_layers` losses
self.history = {'loss':[], 'val_loss':[], 'test_loss':[]}
self.history.update(**pre_suf_fix_dict(self.history, 'step', prefix=True))
# merging the `num_output_layers` histories into one history dictionary
for hist_dict in history_ol_lists:
    self.history.update(**hist_dict)

self.hist_keys = list(self.history.keys())